# extracting the data

In [338]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [339]:
url=[]
for k in range(100):
    url.append('https://www.isna.ir/page/archive.xhtml?mn=12&wide=0&dy=14&ms=0&pi='+str(k)+'&yr=1400')

In [340]:
head=[]
for i in range(len(url)):
    response = requests.get(url[i]).text
    soup = BeautifulSoup(response, 'lxml')
    headlines =soup.find_all('h3')
    for x in headlines[4:]:
        head.append(x.text)

In [341]:
len(head)

3000

## store data in a data frame

In [342]:
news = pd.DataFrame({'title' : head})
news.head()

,title
0,۲۷۷ پایگاه آماده جمع‌آوری کمک‌های مردمی لرستان...
1,تجلیل وزیر فرهنگ لبنان از رایزن ایران و رهبر ا...
2,۱۰۰ فقره جهیزیه به نوعروسان نیازمند لرستانی اه...
3,ایلام واحد تولید پایدار ندارد/خاموشی ادامه ندا...
4,دومین دیدار رافائل گروسی از موزه ملی ایران


## save data to csv

In [343]:
news.to_csv("news.csv", index= False)

## save data to txt

In [344]:
news.to_csv("news_text.txt",  header=None,index= False)

## split data to train and test

In [455]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(news, test_size=0.1, random_state=0)

In [456]:
train.head()

,title
820,فراوانی زندانیان غیربومی و جرم مواد مخدر در خر...
418,«اتل و متل و کوتوله» برای بچه‌ها
704,حضور ورزشکاران خراسان رضوی در اردوی تیم ملی بو...
2589,یارانه نقدی پردرآمدها در سال ۱۴۰۱ حذف می شود
962,تاثیر گرمای محیط در کاهش میزان توجه افراد


In [457]:
test.head()

,title
311,"""طرح عیدانه"" پلیس راهور برای ترخیص خودروهای تو..."
1025,آتش سوزی تالاب انزلی مهار شد/ خسارت ۲۵ هکتاری ...
1587,توسعه همراه با آلودگی ممنوع است
2941,استولتنبرگ: ناتو در اوکراین نیرو مستقر نخواهد کرد
2980,۳۰ میلیارد تومان تسهیلات قرض الحسنه به شهرستان...


# preprocessing

In [458]:
import re
import collections
import math
import random

In [459]:
textfile = open("frequent.txt", "w")

In [460]:
class DataProcessor:
    def __init__(self,data_path):
        self.data_path=data_path
        
        self.data=self.read_data(self.data_path)
        self.cleaned_data=self.clean_text(self.data)
        self.tokenized_data=self.tokenizer(self.cleaned_data)
        self.n_tokens,self.n_unique_tokens,self.frequencies_dict=self.get_most_freqs(self.tokenized_data)
        self.prepared_data=self.handle_unknown(self.tokenized_data,self.frequencies_dict)
        
    def read_data(self,data_path):
        data=[]
        for i in data_path:
            data.append(i)
        return data
    
    def clean_text(self,data):
        cleaned_data=[]
        for j in data:
            st=re.sub(r"[^۰-۹\ئ\ء\آ\ا-ی\.\؟]"," ",j)
            st=re.sub(r"[۰-۹]+","N",st)
            st="\\s " + st + " \\e"
            cleaned_data.append(st)
        return cleaned_data
    
    def tokenizer(self,cleaned_data):
        tokenized_data=[]
        for k in cleaned_data:
            sp=k.split()
            tokenized_data.append(sp)
        return tokenized_data
    
    def get_most_freqs(self,tokenized_data):
        all_tokens=[]
        for i in tokenized_data:
            for j in i :
                all_tokens.append(j)
        n_tokens=len(all_tokens)
        frequencies = collections.Counter(all_tokens)
        n_unique_tokens=len(frequencies.keys())
        sort=sorted(frequencies.items(), key=lambda item: item[1],reverse=True)
        sort2=[]
        for (a,b) in sort:
            sort2.append(a)
        fr = pd.DataFrame(sort2[:200])
        fr.to_csv("frequent.txt",  header=None,index= False)
        frequencies_dict=dict(sort[:10000])
        return n_tokens,n_unique_tokens,frequencies_dict
    
    def handle_unknown(self,tokenized_data,frequencies_dict):
        for i1 in tokenized_data:
            for i2 in i1:
                if not(i2 in frequencies_dict.keys()):
                    tokenized_data[i1][i2]="unk"
        prepared_data=tokenized_data
        return prepared_data

## an object of DataProcessor class

In [461]:
object1=DataProcessor(train['title'])

In [462]:
object1.data[:5]

['فراوانی زندانیان غیربومی و جرم مواد مخدر در خراسان جنوبی',
 '«اتل و متل و کوتوله» برای بچه\u200cها',
 'حضور ورزشکاران خراسان رضوی در اردوی تیم ملی بوچیای ایران',
 'یارانه نقدی پردرآمدها در سال ۱۴۰۱ حذف می شود',
 'تاثیر گرمای محیط در کاهش میزان توجه افراد']

In [463]:
object1.cleaned_data[:5]

['\\s فراوانی زندانیان غیربومی و جرم مواد مخدر در خراسان جنوبی \\e',
 '\\s  اتل و متل و کوتوله  برای بچه ها \\e',
 '\\s حضور ورزشکاران خراسان رضوی در اردوی تیم ملی بوچیای ایران \\e',
 '\\s یارانه نقدی پردرآمدها در سال N حذف می شود \\e',
 '\\s تاثیر گرمای محیط در کاهش میزان توجه افراد \\e']

In [464]:
object1.tokenized_data[:3]

[['\\s',
  'فراوانی',
  'زندانیان',
  'غیربومی',
  'و',
  'جرم',
  'مواد',
  'مخدر',
  'در',
  'خراسان',
  'جنوبی',
  '\\e'],
 ['\\s', 'اتل', 'و', 'متل', 'و', 'کوتوله', 'برای', 'بچه', 'ها', '\\e'],
 ['\\s',
  'حضور',
  'ورزشکاران',
  'خراسان',
  'رضوی',
  'در',
  'اردوی',
  'تیم',
  'ملی',
  'بوچیای',
  'ایران',
  '\\e']]

In [465]:
object1.n_tokens

32109

In [466]:
object1.n_unique_tokens

5382

In [467]:
object1.frequencies_dict

{'\\s': 2700,
 '\\e': 2700,
 'در': 1149,
 'و': 674,
 'به': 583,
 'N': 519,
 'از': 512,
 'های': 358,
 'برای': 299,
 'می': 278,
 'با': 269,
 'است': 229,
 'را': 209,
 'شد': 189,
 'روسیه': 152,
 'ها': 149,
 'اوکراین': 145,
 'شود': 137,
 'ایران': 132,
 'یک': 97,
 'کشور': 95,
 'ملی': 78,
 'طرح': 73,
 'استان': 72,
 'ای': 71,
 'افزایش': 71,
 'کرد': 67,
 'خراسان': 66,
 'تا': 63,
 'هزار': 61,
 'سال': 60,
 'وزیر': 58,
 'درباره': 57,
 'کاهش': 56,
 'بر': 54,
 'کرونا': 54,
 'دانشگاه': 53,
 'جدید': 53,
 'آغاز': 52,
 'آمریکا': 51,
 'قیمت': 50,
 'رئیس': 50,
 'باید': 50,
 'تهران': 48,
 'برگزار': 48,
 'برگزاری': 46,
 'مردم': 46,
 'اجرای': 46,
 'جنگ': 45,
 'دولت': 45,
 'بین': 45,
 'روز': 44,
 'تولید': 43,
 'نیست': 42,
 'کند': 42,
 'حضور': 41,
 'راه': 41,
 'که': 41,
 'جهانی': 41,
 'بیش': 40,
 'نوروزی': 40,
 'سفر': 39,
 'توسعه': 38,
 'تحریم': 38,
 'هفته': 37,
 'دیدار': 37,
 'چه': 37,
 'پایان': 37,
 'شمالی': 37,
 'جشنواره': 36,
 'بررسی': 36,
 'مجلس': 36,
 'خوزستان': 35,
 'میلیارد': 35,
 'آذربایجان': 35,
 'نو

In [468]:
prepared=object1.prepared_data
prepared[:3]

[['\\s',
  'فراوانی',
  'زندانیان',
  'غیربومی',
  'و',
  'جرم',
  'مواد',
  'مخدر',
  'در',
  'خراسان',
  'جنوبی',
  '\\e'],
 ['\\s', 'اتل', 'و', 'متل', 'و', 'کوتوله', 'برای', 'بچه', 'ها', '\\e'],
 ['\\s',
  'حضور',
  'ورزشکاران',
  'خراسان',
  'رضوی',
  'در',
  'اردوی',
  'تیم',
  'ملی',
  'بوچیای',
  'ایران',
  '\\e']]

# Design an Ngram language model

In [469]:
class NgramLanguageModel:
    def __init__(self,prepared_data):
        self.prepared_data=prepared_data
        
        self.unigram_dict,self.bigram_dict,self.trigram_dict=self.make_ngram_dicts(self.prepared_data)
        
    def make_ngram_dicts(self,prepared_data):
        tokens=[]
        for i in prepared_data:
            for j in i :
                tokens.append(j)
        unigram_dict = collections.Counter(tokens)
        two_tokens=[]
        for i1 in prepared_data:
            for i2 in zip(i1[:-1],i1[1:]):
                two_tokens.append(i2[0]+" "+i2[1])
        bigram_dict = collections.Counter(two_tokens)
        tri_tokens=[]
        for j1 in prepared_data:
            for j2 in zip(j1[:-2],j1[1:-1],j1[2:]):
                tri_tokens.append(j2[0]+" "+j2[1]+" "+j2[2])
        trigram_dict = collections.Counter(tri_tokens)
        return unigram_dict,bigram_dict,trigram_dict
    
    def calculate_smoothed_probs(self,n,input):
        most_probs=[]
        st=[]
        st=input.split()
        if len(st)>=1:
            last=st[-1]
            if len(st)>=2:
                second_last=st[-2]

        if(n==2):
            for key in self.unigram_dict.keys():
                if(last+" "+key in self.bigram_dict.keys()):
                    p=(self.bigram_dict.get(last+" "+key)+1)/ (self.unigram_dict.get(last)+len(self.unigram_dict.keys()))
                else:
                    if(last in self.unigram_dict.keys()):
                        p=1/ (self.unigram_dict.get(last)+len(self.unigram_dict.keys()))
                    else:
                        p=1/(len(self.unigram_dict.keys()))
                most_probs.append((key,p))
            most_probs.sort(key=lambda item: (item[1],random.random()),reverse=True)
            most_probs=most_probs[:5]
            return most_probs

        if(n==3):
            for key in self.unigram_dict.keys():
                if(second_last+" "+last+" "+key in self.trigram_dict.keys()):
                    p=(self.trigram_dict.get(second_last+" "+last+" "+key)+1)/ (self.bigram_dict.get(second_last+" "+last)+len(self.unigram_dict.keys()))
                else:
                    if(second_last+" "+last in self.bigram_dict.keys()):
                        p=1/ (self.bigram_dict.get(second_last+" "+last)+len(self.unigram_dict.keys()))
                    else:
                        p=1/ (len(self.unigram_dict.keys()))
                most_probs.append((key,p))
            most_probs.sort(key=lambda item: (item[1],random.random()),reverse=True)
            most_probs=most_probs[:5]
            return most_probs
    def generate_text(self,input,n):
        st2=[]
        st2=input.split()
        a=st2[0]
        count=0
        if(n==2):
            last_char=st2[-1]
            for m in st2[1:]:
                a=a+" "+m
            while (not(last_char=="\\e")) and count<=10:
                last_char=self.calculate_smoothed_probs(2,last_char)[0][0]
                a=a+" "+last_char
                count=count+1
            if (count>10 and not(last_char=="\\e")):
                a=a+" "+"\\e"
        if(n==3):
            last_char1=st2[-1]
            last_char2=st2[-2]
            for m in st2[1:]:
                a=a+" "+m
            while (not(last_char1=="\\e")) and count<=10:
                swap=last_char1
                last_char1=self.calculate_smoothed_probs(3,last_char2+" "+last_char1)[0][0]
                last_char2=swap
                a=a+" "+last_char1
                count=count+1
            if (count>10 and not(last_char1=="\\e")):
                a=a+" "+"\\e"

        return a
    
    def evaluate_model(self,prepared_data):
        avg_log_li2=[]
        avg_log_li3=[]
        for sample in prepared_data:
            log_li2=0
            log_li3=0
            string=sample[0]
            for s in sample[1:6]:
                string=string+" "+s
            string2=self.generate_text(string,2)
            string3=self.generate_text(string,3)
            tokenb=string2.split()
            i=0
            token1=tokenb[i]
            token2=tokenb[i+1]
            while (not(token1=="\\e")):
                if(token1+" "+token2 in self.bigram_dict.keys()):
                    p=(self.bigram_dict.get(token1+" "+token2))/ (self.unigram_dict.get(token1)+len(self.unigram_dict.keys()))
                else:
                    if (token1 in self.unigram_dict.keys()):
                        p=1/ (self.unigram_dict.get(token1)+len(self.unigram_dict.keys()))
                    else:
                        p=1/ (len(self.unigram_dict.keys()))
                log_li2=math.log(p)+log_li2
                i=i+1
                token1=tokenb[i]
                if (not(token1=="\\e")):
                    token2=tokenb[i+1]
            avg_log_li2.append(log_li2 / (len(tokenb)))

            tokent=string3.split()
            i=0
            token1=tokent[i]
            token2=tokent[i+1]
            token3=tokent[i+2]
            while (not(token2=="\\e")):
                if(token1+" "+token2+" "+token3 in self.trigram_dict.keys()):
                    p=(self.trigram_dict.get(token1+" "+token2+" "+token3)+1)/ (self.bigram_dict.get(token1+" "+token2)+len(self.unigram_dict.keys()))
                else:
                    if (token1+" "+token2 in self.bigram_dict.keys()):
                        p=1/ (self.bigram_dict.get(token1+" "+token2)+len(self.unigram_dict.keys()))
                    else:
                        p=1/ (len(self.unigram_dict.keys()))
                log_li3=math.log(p)+log_li3
                i=i+1
                token1=tokent[i]
                token2=tokent[i+1]
                if (not(token2=="\\e")):
                    token3=tokent[i+2]
            avg_log_li3.append(log_li3 / (len(tokent)))
    
        return avg_log_li2,avg_log_li3

    def pp_evaluate_model(self,prepared_data):
        pp2=[]
        pp3=[]
        for sample in prepared_data:
            p2=1
            p3=1
            string=sample[0]
            for s in sample[1:6]:
                string=string+" "+s
            string2=self.generate_text(string,2)
            string3=self.generate_text(string,3)
            tokenb=string2.split()
            i=0
            token1=tokenb[i]
            token2=tokenb[i+1]
            while (not(token1=="\\e")):
                if(token1+" "+token2 in self.bigram_dict.keys()):
                    p=(self.bigram_dict.get(token1+" "+token2)+1)/ (self.unigram_dict.get(token1)+len(self.unigram_dict.keys()))
                else:
                    if (token1 in self.unigram_dict.keys()):
                        p=1/ (self.unigram_dict.get(token1)+len(self.unigram_dict.keys()))
                    else:
                        p=1/ (len(self.unigram_dict.keys()))
                p2=(1/p)*p2
                i=i+1
                token1=tokenb[i]
                if (not(token1=="\\e")):
                    token2=tokenb[i+1]
            pp2.append((p2**(1/(len(tokenb)))))
            
            tokent=string3.split()
            i=0
            token1=tokent[i]
            token2=tokent[i+1]
            token3=tokent[i+2]
            while (not(token2=="\\e")):
                if(token1+" "+token2+" "+token3 in self.trigram_dict.keys()):
                    p=(self.trigram_dict.get(token1+" "+token2+" "+token3)+1)/ (self.bigram_dict.get(token1+" "+token2)+len(self.unigram_dict.keys()))
                else:
                    if (token1+" "+token2 in self.bigram_dict.keys()):
                        p=1/ (self.bigram_dict.get(token1+" "+token2)+len(self.unigram_dict.keys()))
                    else:
                        p=1/ (len(self.unigram_dict.keys()))
                p3=(1/p)*p3
                i=i+1
                token1=tokent[i]
                token2=tokent[i+1]
                if (not(token2=="\\e")):
                    token3=tokent[i+2]
            pp3.append((p3**(1/(len(tokent)))))
    
        return pp2,pp3

## an object of NgramLanguageModel class

In [470]:
object2=NgramLanguageModel(prepared)

In [471]:
object2.unigram_dict

Counter({'\\s': 2700,
         'فراوانی': 1,
         'زندانیان': 4,
         'غیربومی': 1,
         'و': 674,
         'جرم': 2,
         'مواد': 16,
         'مخدر': 4,
         'در': 1149,
         'خراسان': 66,
         'جنوبی': 22,
         '\\e': 2700,
         'اتل': 1,
         'متل': 1,
         'کوتوله': 1,
         'برای': 299,
         'بچه': 3,
         'ها': 149,
         'حضور': 41,
         'ورزشکاران': 5,
         'رضوی': 19,
         'اردوی': 10,
         'تیم': 32,
         'ملی': 78,
         'بوچیای': 1,
         'ایران': 132,
         'یارانه': 5,
         'نقدی': 2,
         'پردرآمدها': 1,
         'سال': 60,
         'N': 519,
         'حذف': 13,
         'می': 278,
         'شود': 137,
         'تاثیر': 3,
         'گرمای': 1,
         'محیط': 4,
         'کاهش': 56,
         'میزان': 6,
         'توجه': 10,
         'افراد': 7,
         'درخواست': 25,
         'بایدن': 13,
         'از': 512,
         'اسرائیل': 8,
         'اتخاذ': 1,
         'موضع': 3,
   

In [472]:
object2.bigram_dict

Counter({'\\s فراوانی': 1,
         'فراوانی زندانیان': 1,
         'زندانیان غیربومی': 1,
         'غیربومی و': 1,
         'و جرم': 1,
         'جرم مواد': 1,
         'مواد مخدر': 4,
         'مخدر در': 4,
         'در خراسان': 19,
         'خراسان جنوبی': 16,
         'جنوبی \\e': 9,
         '\\s اتل': 1,
         'اتل و': 1,
         'و متل': 1,
         'متل و': 1,
         'و کوتوله': 1,
         'کوتوله برای': 1,
         'برای بچه': 1,
         'بچه ها': 1,
         'ها \\e': 14,
         '\\s حضور': 13,
         'حضور ورزشکاران': 2,
         'ورزشکاران خراسان': 1,
         'خراسان رضوی': 15,
         'رضوی در': 6,
         'در اردوی': 2,
         'اردوی تیم': 5,
         'تیم ملی': 16,
         'ملی بوچیای': 1,
         'بوچیای ایران': 1,
         'ایران \\e': 19,
         '\\s یارانه': 1,
         'یارانه نقدی': 1,
         'نقدی پردرآمدها': 1,
         'پردرآمدها در': 1,
         'در سال': 22,
         'سال N': 18,
         'N حذف': 1,
         'حذف می': 1,
         'می شو

In [473]:
object2.trigram_dict

Counter({'\\s فراوانی زندانیان': 1,
         'فراوانی زندانیان غیربومی': 1,
         'زندانیان غیربومی و': 1,
         'غیربومی و جرم': 1,
         'و جرم مواد': 1,
         'جرم مواد مخدر': 1,
         'مواد مخدر در': 4,
         'مخدر در خراسان': 1,
         'در خراسان جنوبی': 6,
         'خراسان جنوبی \\e': 9,
         '\\s اتل و': 1,
         'اتل و متل': 1,
         'و متل و': 1,
         'متل و کوتوله': 1,
         'و کوتوله برای': 1,
         'کوتوله برای بچه': 1,
         'برای بچه ها': 1,
         'بچه ها \\e': 1,
         '\\s حضور ورزشکاران': 2,
         'حضور ورزشکاران خراسان': 1,
         'ورزشکاران خراسان رضوی': 1,
         'خراسان رضوی در': 4,
         'رضوی در اردوی': 1,
         'در اردوی تیم': 1,
         'اردوی تیم ملی': 5,
         'تیم ملی بوچیای': 1,
         'ملی بوچیای ایران': 1,
         'بوچیای ایران \\e': 1,
         '\\s یارانه نقدی': 1,
         'یارانه نقدی پردرآمدها': 1,
         'نقدی پردرآمدها در': 1,
         'پردرآمدها در سال': 1,
         'در سال N':

In [474]:
object2.calculate_smoothed_probs(2,'\\s حقوق')

[('بشر', 0.0009260974254491572),
 ('N', 0.0005556584552694943),
 ('بشری', 0.0005556584552694943),
 ('اعضای', 0.0003704389701796629),
 ('اسفندماه', 0.0003704389701796629)]

In [475]:
object2.calculate_smoothed_probs(3,'\\s حقوق')

[('دولتی', 0.0003714710252600297),
 ('N', 0.0003714710252600297),
 ('فرمانداری', 0.00018573551263001485),
 ('سیدمحسن', 0.00018573551263001485),
 ('نشانگر', 0.00018573551263001485)]

In [476]:
object2.generate_text('\\s فراوانی زندانیان غیربومی و جرم',2)

'\\s فراوانی زندانیان غیربومی و جرم مواد مخدر در استان مرکزی \\e'

In [477]:
object2.generate_text('\\s فراوانی زندانیان غیربومی و جرم',3)

'\\s فراوانی زندانیان غیربومی و جرم مواد مخدر در هفته منابع طبیعی در استان یزد \\e'

In [478]:
object2.generate_text('\\s ادعای نماینده پارلمان',2)

'\\s ادعای نماینده پارلمان عراق \\e'

In [479]:
object2.generate_text('\\s ادعای نماینده پارلمان',3)

'\\s ادعای نماینده پارلمان اوکراین درباره اعتیاد زلنسکی به کوکائین \\e'

In [491]:
object2.evaluate_model([['\\s','فراوانی','زندانیان','غیربومی','و','جرم'],['\\s','ادعای','نماینده','پارلمان']])

([-7.009470276255655, -6.545776796658184],
 [-6.3902348461610545, -6.4619649089902085])

In [498]:
object2.pp_evaluate_model([['\\s','فراوانی','زندانیان','غیربومی','و','جرم'],['\\s','ادعای','نماینده','پارلمان']])

([667.6097740000884, 488.3691194996907],
 [511.7379638369086, 640.3179878582894])

In [499]:
object3=DataProcessor(test['title'])
prepared_test_data=object3.prepared_data
prepared_test_data

[['\\s',
  'طرح',
  'عیدانه',
  'پلیس',
  'راهور',
  'برای',
  'ترخیص',
  'خودروهای',
  'توقیفی',
  'در',
  'استان',
  'کهگیلویه',
  'و',
  'بویراحمد',
  '\\e'],
 ['\\s',
  'آتش',
  'سوزی',
  'تالاب',
  'انزلی',
  'مهار',
  'شد',
  'خسارت',
  'N',
  'هکتاری',
  'حاشیه',
  'تالاب',
  '\\e'],
 ['\\s', 'توسعه', 'همراه', 'با', 'آلودگی', 'ممنوع', 'است', '\\e'],
 ['\\s',
  'استولتنبرگ',
  'ناتو',
  'در',
  'اوکراین',
  'نیرو',
  'مستقر',
  'نخواهد',
  'کرد',
  '\\e'],
 ['\\s',
  'N',
  'میلیارد',
  'تومان',
  'تسهیلات',
  'قرض',
  'الحسنه',
  'به',
  'شهرستان',
  'درمیان',
  'اختصاص',
  'می',
  'یابد',
  '\\e'],
 ['\\s',
  'N',
  'درصد',
  'عرصه',
  'های',
  'ملی',
  'آذربایجان',
  'غربی',
  'سند',
  'کاداستر',
  'دارند',
  '\\e'],
 ['\\s',
  'با',
  'آمدن',
  'یک',
  'روزه',
  'مربی',
  'تغییر',
  'تاکتیکی',
  'رخ',
  'نمی',
  'دهد',
  'بازیکنان',
  'تراکتور',
  'از',
  'فشار',
  'روحی',
  'و',
  'روانی',
  'آزاد',
  'شدند',
  '\\e'],
 ['\\s',
  'روسیه',
  'ماهواره',
  'های',
  'انگلیسی',
 

In [500]:
len(prepared_test_data)

300

In [501]:
av_2gram=object2.evaluate_model(prepared_test_data[:300])[0]
av_2gram

[-6.362066128417235,
 -6.304666619938367,
 -6.317986581932621,
 -6.498386469077242,
 -5.649250918091108,
 -6.077471376218926,
 -6.76883358756708,
 -6.3520171149006455,
 -6.555787854371503,
 -6.493127419638917,
 -7.549500798072762,
 -6.471845251152495,
 -7.264911026565656,
 -6.501864476299189,
 -7.123394262106953,
 -6.492346205393814,
 -6.460112417589502,
 -6.608220563621535,
 -7.37838310358755,
 -6.3715120986727545,
 -7.252662521079259,
 -6.8787315705992045,
 -6.941568788280481,
 -7.339593788139101,
 -6.933462935033795,
 -6.35465193676915,
 -6.631615151748673,
 -6.90668555397895,
 -6.625824718505223,
 -6.943321122213386,
 -6.2592781376259765,
 -6.656446297805922,
 -7.227294945171199,
 -6.133017706036795,
 -7.033517761323454,
 -6.6756600126455305,
 -6.256383128589614,
 -6.891615820862545,
 -6.733932386406132,
 -7.10729024346889,
 -7.410685008309413,
 -6.2062644655660115,
 -6.691753075233885,
 -6.485068737225614,
 -6.488728561913999,
 -6.2054320610205975,
 -6.542732646594835,
 -6.8742459

In [502]:
av_3gram=object2.evaluate_model(prepared_test_data[:300])[1]
av_3gram

[-6.355271900988992,
 -7.53688252056784,
 -7.636383469384317,
 -7.5980297520651545,
 -6.1873135464209765,
 -6.198363469656369,
 -7.636414347793139,
 -7.636393725813331,
 -7.067902798232829,
 -6.526208718939283,
 -7.636290615992427,
 -5.879834076663001,
 -7.636280294477199,
 -7.528739588029205,
 -7.636352524001165,
 -7.636342194824055,
 -7.636424621479467,
 -7.636280294477199,
 -7.636280294477199,
 -6.485810793098198,
 -7.636362812955877,
 -5.979564316780495,
 -6.360943576229804,
 -7.636280294477199,
 -6.183274199392212,
 -7.637184376146355,
 -7.636290615992427,
 -7.636290615992427,
 -7.636311253271846,
 -6.142857818902963,
 -5.528778369140032,
 -7.636280294477199,
 -7.636321578620862,
 -7.508976910553908,
 -6.8127486388003335,
 -7.575339129922546,
 -6.692133446916415,
 -7.636280294477199,
 -7.636280294477199,
 -7.636414366926538,
 -7.636311257105634,
 -6.930163955143443,
 -7.0304463149547045,
 -7.636342204403188,
 -7.636465788991951,
 -7.405427258769281,
 -7.0979079752378285,
 -7.63630

In [505]:
pp_2gram=object2.pp_evaluate_model(prepared_test_data[:300])[0]
pp_2gram

[479.14110843954313,
 509.79588454887147,
 516.6782583917221,
 513.2818118158882,
 250.17066495031756,
 389.90772799512513,
 974.193593391908,
 561.1001079548781,
 443.8007962207244,
 579.605223245839,
 1455.024441017338,
 577.1993905192659,
 1279.4325867002838,
 598.7510175250445,
 1028.0145375594534,
 634.1671006993876,
 572.0443646770781,
 731.9928269415097,
 1435.0421655467644,
 696.238319884908,
 1283.0420485174948,
 871.7396005336793,
 852.4677154995934,
 1349.9972377081112,
 725.0749964640895,
 427.9676718058848,
 699.3747743954888,
 884.730658964999,
 720.9544220356693,
 391.81726999735366,
 264.2996491041981,
 733.6386953712466,
 991.7023493453365,
 407.1161238808527,
 884.3205497542832,
 671.8440732566331,
 435.8766567434958,
 920.0494228366938,
 741.1382510770694,
 675.6882560794043,
 784.7902781866882,
 439.5951613840143,
 562.7679971584487,
 577.1859306251988,
 976.5277108066532,
 440.1394910739307,
 533.6116083751856,
 561.5234672641008,
 662.6346110690316,
 488.447155586

In [506]:
pp_3gram=object2.pp_evaluate_model(prepared_test_data[:300])[1]
pp_3gram

[575.5188035191501,
 1875.9725937193257,
 2072.2359437949763,
 1994.262829597777,
 542.6537687232665,
 542.3630673194327,
 2072.299932131553,
 2072.2571976448316,
 656.4850054153584,
 379.3126381899235,
 2072.043538591678,
 357.7498777305302,
 2072.022152073112,
 1860.7587023558258,
 2072.171818651912,
 2072.150414932735,
 2072.3212224003933,
 2072.022152073112,
 2072.022152073112,
 783.2734811155793,
 2072.1931392435904,
 395.26811894837294,
 578.7922334083761,
 2072.022152073112,
 484.5759600796456,
 2073.896276371945,
 2072.043538591678,
 2072.043538591678,
 2072.086300374393,
 465.381648725198,
 251.8360723115728,
 778.8625404755711,
 2072.1076954990917,
 1824.3461184486175,
 909.3668960866678,
 1949.5212915803145,
 806.0400627499278,
 2072.022152073112,
 2072.022152073112,
 2072.299971781692,
 2072.0863083183326,
 1145.9986150268885,
 824.6907557136507,
 2072.1504347821383,
 2072.40653646626,
 1644.8874785608646,
 1107.7661775957665,
 2072.06492135832,
 2072.1717789600657,
 443.29